In [7]:
import os
from bs4 import BeautifulSoup
import pandas as pd

data = []

# Replace 'your_data_directory' with the actual path to your data directory
data_directory = "./data"

for filename in os.listdir(data_directory):
    if filename.endswith(".txt"):
        with open(os.path.join(data_directory, filename), 'r', encoding='ISO-8859-1') as file:
            html_content = file.read()

            # Extract relevant information using BeautifulSoup methods
            soup = BeautifulSoup(html_content, 'html.parser')

            # Title Information
            title = soup.title.text.strip()

            # Check if the expected patterns exist in the title
            if '|' in title and 'موديل سنة' in title:
                make, model_year = title.split('|')[0].strip().split('موديل سنة')
            else:
                # Handle cases where the expected patterns are not found
                make, model_year = None, None

                # Price Information
                price = soup.find('h5', class_='post-price').text.strip()

                # Features
                features_table = soup.find('table', class_='list_ads')
                features_rows = features_table.find_all('tr', class_='list-row')
                features_dict = {row.find('td').text.strip(): row.find_all('td')[1].text.strip() for row in features_rows}

                # Additional Information
                additional_info_table = soup.find('table', class_='create_post')
                additional_info = additional_info_table.find('td', class_='list-additions').text.strip()

                # Insurance Prices
                insurance_table = soup.find('table', {'class': 'list_ads', 'width': '100%'})
                insurance_rows = insurance_table.find_all('tr')[1:-1]  # Exclude the header and footer rows
                insurance_dict = {row.find('td').text.strip(): row.find_all('td')[1].text.strip() for row in insurance_rows}

                # Advertiser Information
                advertiser_info_table = soup.find('table', {'class': 'create_post', 'width': '100%'})
                advertiser_name = advertiser_info_table.find('td', class_='list-row').text.strip()

                # Contact Information
                contact_info_table = soup.find('table', {'class': 'create_post', 'width': '100%'})
                if contact_info_table:
                    contact_rows = contact_info_table.find_all('tr', class_='list-row')
                    contact_dict = {row.find('td').text.strip(): row.find_all('td')[1].text.strip() for row in contact_rows}
                else:
                    contact_dict = {}

                # Publication Dates
                publish_date_row = soup.find('tr', class_='list-row')
                expiry_date_row = soup.find('tr', {'class': 'list-row', 'id': 'row_1'})

                publish_date = publish_date_row.find_all('td')[1].text.strip() if publish_date_row else None
                expiry_date = expiry_date_row.find_all('td')[1].text.strip() if expiry_date_row else None


                # Store the information in a dictionary
                car_info = {
                    'Make': make,
                    'Model_Year': model_year,
                    'Price': price,
                    **features_dict,
                    'Additional_Info': additional_info,
                    **insurance_dict,
                    'Advertiser_Name': advertiser_name,
                    **contact_dict,
                    'Publish_Date': publish_date,
                    'Expiry_Date': expiry_date
                }

                data.append(car_info)

# Convert the list of dictionaries to a Pandas DataFrame
df = pd.DataFrame(data)


AttributeError: 'NoneType' object has no attribute 'text'